## exp4. 인공지능으로 작사하기

커널정보 = base(python. 3.9.7)

### 0. 루브릭
***

|**평가문항**|**상세기준**|
|------------|-------------|
|1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?|텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?|
|2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?|특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?|
|3. 텍스트 생성모델이 안정적으로 학습되었는가?|텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?|

### 1. 데이터 다운로드 및 모듈 import

### 2. 데이터 읽어오기

In [8]:
import re    
import glob
import numpy as np         
import tensorflow as tf    

txt_file_path = 'C:/project/aiffel_exp/exp4_lyricist/song_lyrics/*.txt'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding="UTF-8") as f:
        raw = f.read().splitlines() # 코드를 그대로 사용하니 unicodedecodeerror 발생하여 인코딩 추가
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


### 3. 데이터 정제

In [10]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


#### 문장 전처리

In [11]:
# 입력된 문장을
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [12]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

#### 데이터 토큰화

In [13]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus) # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
	
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, 
                                                           padding='post',
                                                           maxlen=maxlen)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  23  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  23  77 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x0000019BFEDD5610>


In [14]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [15]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4811    3    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[ 304   28   99 4811    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


### 4. 평가 데이터셋 분리

In [16]:
# 텐서플로우의 `tf.data.Dataset`을 이용해 데이터셋 객체 생성.
# 텐서를 사용할 땐 텐서로 생성된 데이터를 이용해 `tf.data.Dataset` 객체 생성을 주로 사용함
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

### 5. 인공지능 만들기

In [17]:
'''
tk.keras.Model을 Subclassing 하는 방식
1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어 구성
Embedding 레이어는 단어 사전 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔줌. 
이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용됨

embedding_size는 워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기.
값이 커질수록 단어의 추상적인 특징을 잘 잡아내지만, 충분한 데이터가 없다면 오히려 성능이 나빠진다

LSTM레이어의 hidden_size도 같은 맥락
'''
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [18]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 7001), dtype=float32, numpy=
array([[[ 4.03772108e-04, -7.92911596e-05,  2.08476660e-04, ...,
         -7.89116311e-05, -1.51083455e-04,  9.79088654e-05],
        [ 9.13622382e-04, -4.79631679e-04,  1.73344772e-04, ...,
         -1.44497972e-04, -2.64171307e-04,  3.60349659e-04],
        [ 1.06809335e-03, -7.29580817e-04,  3.22581647e-04, ...,
         -4.49454441e-04, -3.23220098e-04,  2.98170489e-04],
        ...,
        [ 2.86653475e-03,  1.61306176e-04, -2.74053495e-03, ...,
         -1.75624271e-03,  4.78496950e-04, -4.69478575e-04],
        [ 3.18243727e-03,  9.12878459e-05, -2.73964740e-03, ...,
         -1.94867188e-03,  4.79712151e-04, -3.24973342e-04],
        [ 3.46260611e-03,  2.75620732e-05, -2.72574322e-03, ...,
         -2.10870733e-03,  4.65457910e-04, -1.75570065e-04]],

       [[-2.52522994e-04, -1.44058242e-04,  2.42146954e-04, ...,
          7.85455995e-05,  3.71460861e-04, -8.30123972e-05],
        [ 3.89929701e-05, -5.50479919e-04,  3.

In [19]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [20]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
686/686 [==============================] - 454s 658ms/step - loss: 2.7139
Epoch 2/30
686/686 [==============================] - 488s 711ms/step - loss: 2.3384
Epoch 3/30
686/686 [==============================] - 272s 396ms/step - loss: 2.1966
Epoch 4/30
686/686 [==============================] - 86s 126ms/step - loss: 2.0910
Epoch 5/30
686/686 [==============================] - 82s 120ms/step - loss: 2.0007
Epoch 6/30
686/686 [==============================] - 82s 119ms/step - loss: 1.9184
Epoch 7/30
686/686 [==============================] - 160s 234ms/step - loss: 1.8414
Epoch 8/30
686/686 [==============================] - 523s 762ms/step - loss: 1.7682
Epoch 9/30
686/686 [==============================] - 498s 726ms/step - loss: 1.6979
Epoch 10/30
686/686 [==============================] - 502s 731ms/step - loss: 1.6292
Epoch 11/30
686/686 [==============================] - 233s 339ms/step - loss: 1.5628
Epoch 12/30
686/686 [==============================] - 396s 577ms/

In [31]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=200):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [45]:
generate_text(model, tokenizer, init_sentence="<start> how")

'<start> how many times i gotta tell that ass to come over ? <end> '